# Rega automática de plantas


![Alt Text](https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExY3Boc3lzNDE0ZDBhc2phM2M1MHIwcGttdHpmcDh4NXFwMzRkOHZxdiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AwfU1IADWoQMTjlAHW/giphy.webp)
![Um relé ligando, ligando uma bomba enviando água para uma planta](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/strawberry-pump.gif?raw=true)

## Introdução

Na última lição, você aprendeu como monitorar a umidade do solo. Nesta lição, você aprenderá como construir os principais componentes de um sistema de irrigação automatizado que responde à umidade do solo. Você também aprenderá sobre tempo - como os sensores podem levar um tempo para responder às mudanças e como os atuadores podem levar um tempo para alterar as propriedades medidas pelos sensores.

Nesta lição abordaremos:

* [Controle dispositivos de alta potência a partir de um dispositivo IoT de baixa potência](#)
* [Controlar relé](#)
* [Controle sua planta via MQTT](#)
* [Sincronização do sensor e do atuador](#)
* [Adicione tempo ao seu servidor de controle de planta](#)



## Controle dispositivos de alta potência a partir de um dispositivo IoT de baixa potência

* Dispositivos IoT usam uma voltagem baixa. 
    * Embora isso seja suficiente para sensores e atuadores de baixa potência como LEDs, isso é muito baixo para controlar hardware maior, como uma bomba de água usada para irrigação. 
    * Mesmo bombas pequenas que você poderia usar para plantas domésticas consomem muita corrente para um kit de desenvolvimento IoT e queimariam a placa.

> 🎓 Corrente, medida em Amps (A), é a quantidade de eletricidade que se move através de um circuito. A voltagem fornece o empurrão, a corrente é o quanto é empurrado. Você pode ler mais sobre corrente na [página de corrente elétrica na Wikipedia](https://wikipedia.org/wiki/Electric_current).

* A solução para isso é ter uma bomba conectada a uma fonte de alimentação externa e usar um atuador para ligar a bomba 
    * Semelhante a como você ligaria uma luz. 
    * É preciso uma pequena quantidade de energia (na forma de energia em seu corpo) para que seu dedo acione um interruptor, e isso conecta a luz à eletricidade da rede elétrica funcionando a 110v/240v.

![Um interruptor de luz liga a energia de uma luz](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/light-switch.png?raw=true)

> 🎓 [Eletricidade da rede elétrica](https://wikipedia.org/wiki/Eletricidade_da_rede) refere-se à eletricidade fornecida a residências e empresas por meio de infraestrutura nacional em muitas partes do mundo.

✅ Dispositivos IoT geralmente podem fornecer 3,3 V ou 5 V, a menos de 1 amp (1 A) de corrente. Compare isso com a eletricidade da rede elétrica, que geralmente é de 230 V (120 V na América do Norte e 100 V no Japão) e pode fornecer energia para dispositivos que consomem 30 A.

Há uma série de atuadores que podem fazer isso, incluindo dispositivos mecânicos que você pode anexar a interruptores existentes que imitam um dedo ligando-os. O mais popular é um relé.



### Relé

Um relé é um interruptor eletromecânico que converte um sinal elétrico em um movimento mecânico que liga um interruptor. O núcleo de um relé é um eletroímã.

> 🎓 [Eletroímãs](https://wikipedia.org/wiki/Electromagnet) são ímãs criados pela passagem de eletricidade por uma bobina de fio. Quando a eletricidade é ligada, a bobina fica magnetizada. Quando a eletricidade é desligada, a bobina perde seu magnetismo.

![Quando ligado, o eletroímã cria um campo magnético, ligando o interruptor do circuito de saída](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/relay-on.png?raw=true)

Em um relé, um circuito de controle energiza o eletroímã. Quando o eletroímã está ligado, ele puxa uma alavanca que move um interruptor, fechando um par de contatos e completando um circuito de saída.

![Quando desligado, o eletroímã não cria um campo magnético, desligando o interruptor do circuito de saída](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/relay-off.png?raw=true)

Quando o circuito de controle está desligado, o eletroímã desliga, liberando a alavanca e abrindo os contatos, desligando o circuito de saída. Relés são atuadores digitais - um sinal alto para o relé o liga, um sinal baixo o desliga.

* O circuito de saída pode ser usado para alimentar hardware adicional, **como um sistema de irrigação**. 
    1. O dispositivo IoT pode ligar o relé, completando o circuito de saída que alimenta o sistema de irrigação, e as plantas são regadas. 
    1. O dispositivo IoT pode então desligar o relé, cortando a energia do sistema de irrigação, desligando a água.

![Um relé ligando, ligando uma bomba enviando água para uma planta](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/strawberry-pump.gif?raw=true)

No vídeo acima, um relé é ligado. Um LED no relé acende para indicar que ele está ligado (algumas placas de relé têm LEDs para indicar se o relé está ligado ou desligado), e a energia é enviada para a bomba, ligando-a e bombeando água para uma planta.

> 💁 Relés também podem ser usados ​​para alternar entre dois circuitos de saída em vez de ligar e desligar um. Conforme a alavanca se move, ela move um interruptor de completar um circuito de saída para completar um circuito de saída diferente, geralmente compartilhando uma conexão de energia comum ou conexão de aterramento comum.

✅ Faça alguma pesquisa: Existem vários tipos de relés, com diferenças como se o circuito de controle liga ou desliga o relé quando a energia é aplicada, ou vários circuitos de saída. Descubra sobre esses diferentes tipos.

Quando a alavanca se move, normalmente você pode ouvi-la fazendo contato com o eletroímã com um ruído de clique bem definido.

> 💁 Um relé pode ser conectado de forma que fazer a conexão realmente interrompa a energia para o relé, desligando o relé, que então envia energia para o relé ligando-o novamente, e assim por diante. Isso significa que o relé clicará incrivelmente rápido, fazendo um ruído de zumbido. É assim que algumas das primeiras campainhas usadas em campainhas elétricas funcionavam.



### Potência do relé

O eletroímã não precisa de muita energia para ativar e puxar a alavanca, ele pode ser controlado usando a saída de 3,3 V ou 5 V de um kit de desenvolvimento de IoT. O circuito de saída pode transportar muito mais energia, dependendo do relé, incluindo tensão de rede ou até mesmo níveis de energia mais altos para uso industrial. Dessa forma, um kit de desenvolvimento de IoT pode controlar um sistema de irrigação, desde uma pequena bomba para uma única planta até um sistema industrial massivo para uma fazenda comercial inteira.

![Um relé Grove com o circuito de controle, circuito de saída e relé rotulados](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/grove-relay-labelled.png?raw=true)

A imagem acima mostra um relé Grove. O circuito de controle se conecta a um dispositivo IoT e liga ou desliga o relé usando 3,3 V ou 5 V. O circuito de saída tem dois terminais, um pode ser de energia ou terra. O circuito de saída pode suportar até 250 V a 10 A, o suficiente para uma variedade de dispositivos alimentados pela rede elétrica. Você pode obter relés que podem suportar até altos níveis de energia.

![Uma bomba conectada por meio de um relé](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/pump-wired-to-relay.png?raw=true)

Na imagem acima, a energia é fornecida a uma bomba por meio de um relé. 
* Há um fio vermelho conectando o terminal +5V de uma fonte de alimentação USB a um terminal do circuito de saída do relé, e outro fio vermelho conectando o outro terminal do circuito de saída à bomba. 
* Um fio preto conecta a bomba ao aterramento na fonte de alimentação USB. 
* Quando o relé liga, ele completa o circuito, enviando 5V para a bomba, ligando-a.

## Controlar um relé

Você pode controlar um relé a partir do seu kit de desenvolvimento IoT.

### Tarefa - controlar um relé

Siga o guia relevante para controlar um relé usando seu dispositivo IoT:

* [Arduino - Terminal Wio](https://github.com/microsoft/IoT-For-Beginners/blob/main/2-farm/lessons/3-automated-plant-watering/wio-terminal-relay.md)
* [Computador de placa única - Raspberry Pi](https://github.com/microsoft/IoT-For-Beginners/blob/main/2-farm/lessons/3-automated-plant-watering/pi-relay.md)
* [Computador de placa única - Dispositivo virtual](https://github.com/microsoft/IoT-For-Beginners/blob/main/2-farm/lessons/3-automated-plant-watering/virtual-device-relay.md)



# Iremos usar o dispositivo virtual!

# Controlar um relé - Virtual IoT Hardware

Nesta parte da lição, você adicionará um relé ao seu dispositivo IoT virtual, além do sensor de umidade do solo, e o controlará com base no nível de umidade do solo.

## Hardware Virtual

O dispositivo IoT virtual usará um relé Grove simulado. Isso mantém este laboratório igual ao uso de um Raspberry Pi com um relé Grove físico.

* Em um dispositivo físico de IoT, o relé seria um relé normalmente aberto (o que significa que o circuito de saída está aberto ou desconectado quando não há sinal enviado ao relé). 
* Um relé como esse pode lidar com circuitos de saída de até 250 V e 10 A.

### Adicione o relé ao CounterFit

Para usar um revezamento virtual, você precisa adicioná-lo ao aplicativo CounterFit

#### Tarefa

Adicione o relé ao aplicativo CounterFit.

1. Abra o projeto `soil-moisture-sensor` da última lição no VS Code, se ele ainda não estiver aberto. Você estará adicionando a este projeto.


In [1]:
from counterfit_connection import CounterFitConnection
CounterFitConnection.init('127.0.0.1', 5001)

import time
from counterfit_shims_grove.adc import ADC

adc = ADC()

while True:
    soil_moisture = adc.read(0)
    print("Soil moisture:", soil_moisture)

    time.sleep(5)

Soil moisture: 266


KeyboardInterrupt: 


2. Certifique-se de que o aplicativo da web CounterFit esteja em execução

1. Crie um relé:

    1. Na caixa *Criar atuador* no painel *Atuadores*, abra a caixa *Tipo de atuador* e selecione *Relé*.

    1. Defina o *Pin* como *5*

    1. Selecione o botão **Adicionar** para criar o relé no pino 5

    ![As configurações do relé](https://github.com/microsoft/IoT-For-Beginners/raw/main/images/counterfit-create-relay.png)

    O relé será criado e aparecerá na lista de atuadores.

    ![O relé criado](https://github.com/microsoft/IoT-For-Beginners/raw/main/images/counterfit-relay.png)



## Programe o relé

O aplicativo sensor de umidade do solo agora pode ser programado para usar o relé virtual.

### Tarefa

Programe o dispositivo virtual.

1. Abra o projeto `soil-moisture-sensor` da última lição no VS Code, se ele ainda não estiver aberto. Você estará adicionando a este projeto.

1. Adicione o seguinte código ao arquivo `app.py` abaixo das importações existentes:

    ```python
    from counterfit_shims_grove.grove_relay import GroveRelay
    ```

    Esta declaração importa o `GroveRelay` das bibliotecas shim do Grove Python para interagir com o relé Grove virtual.

1. Adicione o seguinte código abaixo da declaração da classe `ADC` para criar uma instância `GroveRelay`:

    ```python
    relay = GroveRelay(5)
    ```

    Isso cria um relé usando o pino **5**, o pino ao qual você conectou o relé.

1. Para testar se o relé está funcionando, adicione o seguinte ao loop `while True:`:

    ```python
    relay.on()
    time.sleep(.5)
    relay.off()
    ```

    O código liga o relé, aguarda 0,5 segundos e depois desliga o relé.

1. Execute o aplicativo Python. O relé ligará e desligará a cada 10 segundos, com um atraso de meio segundo entre ligar e desligar. Você verá o relé virtual no aplicativo CounterFit fechar e abrir conforme o relé é ligado e desligado.

    ![O relé virtual ligando e desligando](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/virtual-relay-turn-on-off.gif?raw=true)


In [5]:
import time
from counterfit_connection import CounterFitConnection
from counterfit_shims_grove.adc import ADC
from counterfit_shims_grove.grove_relay import GroveRelay

CounterFitConnection.init('127.0.0.1', 5001)

adc = ADC()


relay = GroveRelay(5)


from counterfit_shims_grove.grove_relay import GroveRelay

while True:
    soil_moisture = adc.read(0)
    print("Soil moisture:", soil_moisture)
    
    relay.on()
    time.sleep(2)
    relay.off()
    time.sleep(2)

Soil moisture: 800
Soil moisture: 513


KeyboardInterrupt: 


## Controle o relé da umidade do solo

Agora que o relé está funcionando, ele pode ser controlado em resposta às leituras de umidade do solo.

### Tarefa

Controle o relé.

1. Exclua as 3 linhas de código que você adicionou para testar o relé. Substitua-as pelo seguinte código em seu lugar:

    ```python
    if soil_moisture > 450:
        print("Soil Moisture is too low, turning relay on.")
        relay.on()
    else:
        print("Soil Moisture is ok, turning relay off.")
        relay.off()
    ```

    Este código verifica o nível de umidade do solo pelo sensor de umidade do solo. Se estiver acima de 450, ele liga o relé, desligando-o se ficar abaixo de 450.

    > 💁 Lembre-se de que o sensor capacitivo de umidade do solo lê que quanto menor o nível de umidade do solo, mais umidade há no solo e vice-versa.



In [8]:
import time
from counterfit_connection import CounterFitConnection
from counterfit_shims_grove.adc import ADC
from counterfit_shims_grove.grove_relay import GroveRelay

CounterFitConnection.init('127.0.0.1', 5001)

adc = ADC()


relay = GroveRelay(5)

while True:
    soil_moisture = adc.read(0)
    print("Soil moisture:", soil_moisture)
    
    if soil_moisture > 450:
        print("Soil Moisture is too low, turning relay on.")
        relay.on()
    else:
        print("Soil Moisture is ok, turning relay off.")
        relay.off()
    time.sleep(10)

Soil moisture: 889
Soil Moisture is too low, turning relay on.
Soil moisture: 992
Soil Moisture is too low, turning relay on.


KeyboardInterrupt: 

2. Execute o aplicativo Python. Você verá o relé ligar ou desligar dependendo dos níveis de umidade do solo. Altere as configurações *Value* ou *Random* para o sensor de umidade do solo para ver a mudança de valor.

    ```saída
    Umidade do solo: 638
    A umidade do solo está muito baixa, ligue o relé.
    Umidade do solo: 452
    A umidade do solo está muito baixa, ligue o relé.
    Umidade do solo: 347
    A umidade do solo está ok, desligando o relé.
    ```

> 💁 Você pode encontrar este código na pasta [code-relay/virtual-device](https://github.com/microsoft/IoT-For-Beginners/blob/main/2-farm/lessons/3-automated-plant-watering/code-relay/virtual-device).

😀 Seu sensor virtual de umidade do solo controlando um programa de relé foi um sucesso!